In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import mysql.connector
import pandas as pd
import json

In [4]:
#
# THIS PART CONTAINS ALL SHARED FUNCTIONS ...
#

class Shared:
    @classmethod
    def open_database(self):
        con = mysql.connector.connect(user='dbadmin', password='washywashy',host='127.0.0.1',database='tests')
        con.autocommit = False
        return con, con.cursor()

    @classmethod
    def read_csv(self, filename):
        df = pd.read_csv(filename, low_memory=False)
        return df

    @classmethod
    def select_columns(self, df, columns):
        return df[columns]

    @classmethod
    def get_data(self):
        df = Shared.read_csv('./data/movies_imbd.csv')
        return df

    @classmethod
    def get_column_df(self, col):
        df = Shared.read_csv('./data/movies_imbd.csv')
        df = Shared.select_columns(df,[col])
        df = df[~pd.isnull(df[col])]
        df.fillna("",inplace=True)
        return df
    
    @classmethod
    def get_column_df_movie_id(self, col):
        df = Shared.read_csv('./data/movies_imbd.csv')
        df = Shared.select_columns(df,[col, 'id'])
        df = df[~pd.isnull(df[col])]
        df.fillna("",inplace=True)
        df['movie_id'] = df['id']
        df = df[[col, 'movie_id']]
        return df    

    @classmethod
    def from_key_to_key(self, s, fk, tk):
        fp = s.find("'"+fk+"':")
        tp = s.find("'"+tk+"':")    
        return s[fp+len(fk)+4:tp-2]

    @classmethod
    def get_single_id_name(self, s):
        return Shared.from_key_to_key(s, "id", "name"), Shared.from_key_to_key(s, "name", "poster_path")

In [5]:
#
# THIS IS FOR PRODUCTION COUNTRIES ...
#

class ProductionCountries:

    @classmethod
    def __get_production_countries_data(self):
        df = Shared.get_column_df('production_countries')
        production_countries = {}
        for c in df.values:
            if (c[0] != '[]'):
                lines = c[0].split('},')
                for l in lines:
                    l = l.replace("[{'iso_3166_1","{'iso_3166_1").replace("}]","}")

                    if l[-1] != "}":
                        l = l + "}"
                    l = l.strip()

                    fp = l.find("'"+'iso_3166_1'+"':")
                    tp = l.find(""+'name'+"")
                    initials = l[fp+len('iso_3166_1')+5:tp-4]

                    fp = l.find("'"+'name'+"':")
                    tp = l.find(""+'}'+"")
                    country = l[fp+len('name')+5:tp-1]
                    
                    if initials != '' and country != '':
                        production_countries[initials] = country

        return production_countries   

    @classmethod
    def create_table(self):
        con, cursor = Shared.open_database()
        cursor.execute("drop table if exists tests.production_countries")
        create_table_sql = """
            create table tests.production_countries (
                id int not null auto_increment,
                initials varchar(10),
                name varchar(250),
                primary key(id)
            )
            """
        cursor.execute(create_table_sql)
        con.commit()    
        con.close()

    @classmethod
    def insert_data(self):
        pc = self.__get_production_countries_data()
        values = []
        for key in pc:
            values.append((key,pc[key]))
        con, cursor = Shared.open_database()
        cursor.executemany("insert into tests.production_countries (initials, name) values (%s, %s)", values)
        con.commit()    
        con.close()  

        
# ProductionCountries.create_table()
# ProductionCountries.insert_data()

In [20]:
#
# THIS IS FOR PRODUCTION COMPANIES ...
#

class ProdcutionCompanies:
    
    @classmethod
    def __get_production_companies_data(self):
        production_companies = {}
        movies_production_companies = []
        df = Shared.get_column_df_movie_id('production_companies')
        for c in df.sample(1000).values:
            if(c[0] != '[]'):
                parts = c[0].split('},')        
                for p in parts:
                    p = p.replace("[{'id","{'id").replace(" {'id'", "{'id'").replace("}]", "}").replace("} ", "}")
                    p = p.replace("[{'name","{'name")
                    if p[-1] != '}':
                        p = p + "}"
                        p = p.replace("{'name': ", "").replace("'id': ", "").replace('}', '')        
                        parts2 = p.split("', ")
                        if len(parts2) == 2:
                            name = parts2[0].strip()[1:]
                            id = parts2[1]
                            if id != '' and name != '':
                                production_companies[id] = name
                        else:
                            parts2 = p.split('", ')
                            name = parts2[0].strip()[1:]
                            id = parts2[1]
                            if id != '' and name != '':
                                production_companies[id] = name
                        try:
                            movies_production_companies.append((int(c[1]), id))
                        except: 
                            pass   
                        
        return production_companies, movies_production_companies

    @classmethod
    def create_table(self):
        con, cursor = Shared.open_database()
        cursor.execute("drop table if exists tests.production_companies")
        cursor.execute("drop table if exists tests.movies_production_companies")
        create_table_sql = """
        create table tests.production_companies (
            id int,
            name varchar(512),
            primary key(id)
        )
        """
        cursor.execute(create_table_sql)
        create_table_sql = """
        create table tests.movies_production_companies (
            movie_id int,
            production_company_id int
        )
        """
        cursor.execute(create_table_sql)
        con.commit()    
        con.close()

    @classmethod
    def insert_data(self):
        pc, pcm = self.__get_production_companies_data()
        values = []
        for key in pc:
            values.append((key, pc[key]))
        con, cursor = Shared.open_database()
        cursor.executemany("insert into tests.production_companies (id, name) values (%s, %s)", values)
        con.commit()
        
        con.autocommit = True        
        chunk_size = 1000
        for i in range(0, len(pcm), chunk_size):
            print(i, 'insering to tests.movies_production_companies ...')
            cursor.executemany("insert into tests.movies_production_companies (movie_id, production_company_id) values (%s, %s)", pcm[i:i + chunk_size])
        con.autocommit = False
        con.close() 
        
# ProdcutionCompanies.create_table()
# ProdcutionCompanies.insert_data()

In [34]:
#
# THIS IS FOR MOVIE COLLECTIONS ...
#

class MovieCollections:
    
    @classmethod
    def __get_collection_data(self):
        df = Shared.get_column_df_movie_id("belongs_to_collection")
        collection_belonging = {}
        movies_collections = []
        for s in df.values:
            id = Shared.from_key_to_key(s[0],'id','name')        
            name = Shared.from_key_to_key(s[0],'name','poster_path')
            name = name.replace("'", "").replace('"', '')
            if id != '' and name != '':
                collection_belonging[id] = name
            try:
                movies_collections.append((int(s[1]),id))
            except:
                pass

        return collection_belonging, movies_collections
    
    @classmethod
    def create_table(self):
        con, cursor = Shared.open_database()
        cursor.execute("drop table if exists tests.collections")
        cursor.execute("drop table if exists tests.movie_collections")
        create_table_sql = """
        create table tests.collections (
            id int,
            name varchar(512),
            primary key(id)
        )
        """
        cursor.execute(create_table_sql)
        create_table_sql = """
        create table tests.movie_collections (
            movie_id int,
            collection_id int
        )
        """
        cursor.execute(create_table_sql)
        con.commit()    
        con.close()
        
    @classmethod
    def insert_data(self):
        cb, mcb = self.__get_collection_data()
        con, cursor = Shared.open_database()
        values = []
        for k in cb:
            values.append((k, cb[k]))
        cursor.executemany("insert into tests.collections (id, name) values (%s, %s)", values)
        con.commit() 
        
        con.autocommit = True        
        chunk_size = 1000
        for i in range(0, len(mcb), chunk_size):
            print(i, 'insering to tests.movie_collections ...')
            cursor.executemany("insert into tests.movie_collections (movie_id, collection_id) values (%s, %s)", mcb[i:i + chunk_size])
        con.autocommit = False
        con.close()    
        
# MovieCollections.create_table()
# MovieCollections.insert_data()

In [8]:
#
# THIS IS FOR SPOKEN LANGUAGES ...
#

class SpokenLanguages:
    
    @classmethod
    def __get_spoken_Lnaguage_data(self):
        df = Shared.get_column_df("spoken_languages")
        spoken_languages = {}
        for s in df.values:
            if (s[0] != '[]'):
                lines = s[0].split('},')
                for l in lines:
                    l = l.replace("[{'iso_639_1","{'iso_639_1").replace("}]","}")

                    if l[-1] != "}":
                        l = l + "}"
                    l = l.strip()

                    fp = l.find("'"+'iso_639_1'+"':")
                    tp = l.find(""+'name'+"")
                    initials = l[fp+len('iso_639_1')+5:tp-4]

                    fp = l.find("'"+'name'+"':")
                    tp = l.find(""+'}'+"")

                    name = l[fp+len('name')+5:tp-1]
                    spoken_languages[initials] = name
        return spoken_languages 
    
    @classmethod
    def create_table(self):
        con, cursor = Shared.open_database()
        cursor.execute("drop table if exists tests.spoken_languages")
        create_table_sql = """
        create table tests.spoken_languages (
                id int not null auto_increment,
                initial varchar(10),
                language varchar(250),
                primary key(id)
        )
        """
        cursor.execute(create_table_sql)
        con.commit()    
        con.close()

    @classmethod
    def insert_data(self):
        sl = self.__get_spoken_Lnaguage_data()
        con, cursor = Shared.open_database()
        values = []
        for k in sl:
            values.append((k, sl[k]))
        cursor.executemany("insert into tests.spoken_languages (initial, language) values (%s, %s)", values)
        con.commit()   
        con.close()    
        
# SpokenLanguages.create_table()
# SpokenLanguages.insert_data()

In [9]:
#
# THIS IS FOR GENRES ...
#

class Genres:
    
    @classmethod
    def __add_to_movie_genre(self, movie_id, genre_id):        
        print('movie_id:', movie_id, 'genre_id:', genre_id)
    
    @classmethod
    def __get_genres_data(self):
        genres = {}
        moveies_genre = []
        df = Shared.get_column_df_movie_id('genres')
        for s in df.values:
                if (s[0] != '[]'):
                    parts = s[0].split('},')
                    for p in parts:
                        p = p.replace("[{'id","{'id").replace(" {'id'", "{'id'").replace("}]", "}").replace("} ", "}")
                        p = p.replace("[{'name","{'name")
                        if p[-1] != '}':
                            p = p + "}"
                        p = p.replace("'", '"')
                        obj = json.loads(p)

                        id = obj['id']
                        name = obj['name']
                        if id != '' and name != '':
                            genres[id] = name
                            try:
                                moveies_genre.append((int(s[1]), id))
                            except: 
                                pass                                
                            
                            
        return genres, moveies_genre

    @classmethod
    def create_table(self):
        con, cursor = Shared.open_database()
        cursor.execute("drop table if exists tests.genres")
        cursor.execute("drop table if exists tests.movies_genre")
        create_table_sql = """
            create table tests.genres (
                id int,
                name varchar(512),
                primary key(id)
            )
        """
        cursor.execute(create_table_sql)
        create_table_sql = """
            create table tests.movies_genre (
                movie_id int,
                genre_id int
            )
        """
        cursor.execute(create_table_sql)
        con.commit()    
        con.close()

    @classmethod
    def insert_data(self):
        gens, moveies_genre = self.__get_genres_data()
        values = []
        for key in gens:
            values.append((key,gens[key]))
        con, cursor = Shared.open_database()
        cursor.executemany("insert into tests.genres (id, name) values (%s, %s)", values)
        con.commit()    
        
        con.autocommit = True        
        chunk_size = 1000
        for i in range(0, len(moveies_genre), chunk_size):
            print(i, 'insering to tests.movies_genre ...')
            cursor.executemany("insert into tests.movies_genre (movie_id, genre_id) values (%s, %s)", moveies_genre[i:i + chunk_size])
        con.autocommit = False
        con.close()     

# Genres.create_table()
# Genres.insert_data()    

In [21]:
df = Shared.get_column_df_movie_id("belongs_to_collection")
df

,belongs_to_collection,movie_id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",15602
4,"{'id': 96871, 'name': 'Father of the Bride Col...",11862
9,"{'id': 645, 'name': 'James Bond Collection', '...",710
12,"{'id': 117693, 'name': 'Balto Collection', 'po...",21032
...,...,...
45355,"{'id': 37261, 'name': 'The Carry On Collection...",24568
45358,"{'id': 37261, 'name': 'The Carry On Collection...",19307
45369,"{'id': 37261, 'name': 'The Carry On Collection...",21251
45371,"{'id': 477208, 'name': 'DC Super Hero Girls Co...",460135
